In [2]:
import sentencepiece as spm
from datasets import load_dataset

dataset = load_dataset("wikimedia/wikipedia", "20231101.hy", split='train', streaming=True)

with open('corpus_large.txt', 'w', encoding='utf-8') as f:
    for i, example in enumerate(dataset):
        sentences = example['text'].split('.')
        for sentence in sentences:
            clean_sentence = sentence.strip()
            if len(clean_sentence) > 20:
                f.write(clean_sentence + "\n")
        if i >= 5000:
            break

spm.SentencePieceTrainer.train(
    input='corpus_large.txt',
    model_prefix='hy_large',
    vocab_size=8000,
    character_coverage=1.0,
    model_type='bpe'
)

sp_small = spm.SentencePieceProcessor(model_file='hy_bpe.model')
sp_large = spm.SentencePieceProcessor(model_file='hy_large.model')

my_sentences = [
    "Հայաստանի հանրապետությունը ունի հազարամյակների պատմություն:",
    "Ժամանակակից տեխնոլոգիաները զարգանում են շատ արագ:",
    "Մենք սովորում ենք մեքենայական ուսուցում և տվյալագիտություն:",
    "Գրքերը գիտելիքի ամենակարևոր աղբյուրն են:",
    "Այս ծրագիրը աշխատում է իմ համակարգչի վրա:"
]

total_small = 0
total_large = 0

for s in my_sentences:
    ids_s = sp_small.encode_as_ids(s)
    ids_l = sp_large.encode_as_ids(s)
    total_small += len(ids_s)
    total_large += len(ids_l)
    
    print(f"Original: {s}")
    print(f"Large Model Pieces: {sp_large.encode_as_pieces(s)}")
    print("-" * 20)

print(f"Average tokens (Small Model): {total_small / 5}")
print(f"Average tokens (Large Model): {total_large / 5}")

Original: Հայաստանի հանրապետությունը ունի հազարամյակների պատմություն:
Large Model Pieces: ['▁Հայաստանի', '▁հանրապետ', 'ությունը', '▁ունի', '▁հազար', 'ամյակ', 'ների', '▁պատմություն', ':']
--------------------
Original: Ժամանակակից տեխնոլոգիաները զարգանում են շատ արագ:
Large Model Pieces: ['▁Ժամանակակից', '▁տեխնոլոգ', 'իաները', '▁զարգ', 'անում', '▁են', '▁շատ', '▁արագ', ':']
--------------------
Original: Մենք սովորում ենք մեքենայական ուսուցում և տվյալագիտություն:
Large Model Pieces: ['▁Մ', 'ենք', '▁սովոր', 'ում', '▁ենք', '▁մեքեն', 'այական', '▁ուսուց', 'ում', '▁եւ', '▁տվյալ', 'ագիտություն', ':']
--------------------
Original: Գրքերը գիտելիքի ամենակարևոր աղբյուրն են:
Large Model Pieces: ['▁Գր', 'քերը', '▁գիտ', 'ելի', 'քի', '▁ամեն', 'ակար', 'եւոր', '▁աղբյուր', 'ն', '▁են', ':']
--------------------
Original: Այս ծրագիրը աշխատում է իմ համակարգչի վրա:
Large Model Pieces: ['▁Այս', '▁ծրագիրը', '▁աշխատում', '▁է', '▁իմ', '▁համակարգ', 'չի', '▁վրա', ':']
--------------------
Average tokens (Small Mo